<a href="https://colab.research.google.com/github/Akashshankar96/BizCardX_Extracting_Business_Card_Data_with_OCR/blob/main/BizCardX_Extracting_Business_Card_Data_with_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import sqlite3
import easyocr
import numpy as np
from PIL import Image
import io
import pandas as pd


# Connect to the database
conn = sqlite3.connect('business_cards.db')
c = conn.cursor()

# Create the business_cards table if it doesn't exist
c.execute('''CREATE TABLE IF NOT EXISTS business_cards
             (id INTEGER PRIMARY KEY AUTOINCREMENT,
              name TEXT,
              phone_number TEXT,
              email TEXT,
              image BLOB)''')


# Function to extract information from an image
def extract_information(image):
    # Read the image and convert it to grayscale
    img = np.array(Image.open(io.BytesIO(image)).convert('L'))

    # Apply thresholding to binarize the image
    threshold = 150
    img[img < threshold] = 0
    img[img >= threshold] = 255

    # Initialize the OCR reader
    reader = easyocr.Reader(['en'])

    # Extract the text from the image using OCR
    results = reader.readtext(img)

    # Extract the relevant information from the OCR results
    info = {}
    for r in results:
        if 'Name' in r[1]:
            info['Name'] = r[0]
        elif 'Phone' in r[1]:
            info['Phone Number'] = r[0]
        elif 'Email' in r[1]:
            info['Email'] = r[0]

    return info


# Function to insert data into the database
def insert_data(info, image):
    name = info.get('Name', '')
    phone_number = info.get('Phone Number', '')
    email = info.get('Email', '')
    c.execute('''INSERT INTO business_cards (name, phone_number, email, image)
                 VALUES (?, ?, ?, ?)''', (name, phone_number, email, image))
    conn.commit()


# Function to retrieve data from the database
def retrieve_data():
    c.execute('''SELECT id, name, phone_number, email FROM business_cards''')
    data = c.fetchall()
    return data


# Function to update data in the database
def update_data(card_id, new_info, image):
    name = new_info.get('Name', '')
    phone_number = new_info.get('Phone Number', '')
    email = new_info.get('Email', '')
    c.execute('''UPDATE business_cards SET name=?, phone_number=?, email=?, image=?
                 WHERE id=?''', (name, phone_number, email, image, card_id))
    conn.commit()


# Function to delete data from the database
def delete_data(card_id):
    c.execute('''DELETE FROM business_cards WHERE id=?''', (card_id,))
    conn.commit()


# Streamlit app
st.set_page_config(page_title="Business Card Reader", page_icon=":credit_card:")
st.title("Business Card Reader")

# Upload an image file
uploaded_file = st.file_uploader("Choose a business card image", type=['jpg', 'jpeg', 'png'])

# Display the extracted information
if uploaded_file is not None:
    # Extract the information from the image
    image = uploaded_file.read()
    info = extract_information(image)

    # Insert the information into the database
    insert_data(info, image)

    # Display the information
    st.header("Extracted Information")
    st.write("Name: ", info.get('Name', ''))
    st.write("Phone Number: ", info.get('Phone Number', ''))
    st.write("Email: ", info.get('Email', ''))

   

    # Display the data in a table
    data = retrieve_data()
    if len(data) > 0:
        st.header("Business Cards in the Database")
        columns = ['ID', 'Name', 'Phone Number', 'Email']
        df = pd.DataFrame(data, columns=columns)
        st.table(df)

    # Allow the user to update or delete the data
    st.header("Update or Delete a Business Card")
    card_id = st.text_input("Enter the ID of the business card you want to update or delete:")
    if card_id:
        card = [x for x in data if x[0] == int(card_id)]
        if len(card) > 0:
            card = card[0]
            st.write("Name: ", card[1])
            st.write("Phone Number: ", card[2])
            st.write("Email: ", card[3])
            action = st.radio("Select an action:", ("Update", "Delete"))
            if action == "Update":
                new_name = st.text_input("Enter the new name:", value=card[1])
                new_phone_number = st.text_input("Enter the new phone number:", value=card[2])
                new_email = st.text_input("Enter the new email:", value=card[3])
                new_info = {'Name': new_name, 'Phone Number': new_phone_number, 'Email': new_email}
                update_data(card[0], new_info, image)
                st.success("Business card updated successfully!")
            elif action == "Delete":
                delete_data(card[0])
                st.success("Business card deleted successfully!")
        else:
            st.warning("Business card not found.")



  command:

    streamlit run /usr/local/lib/python3.8/dist-packages/ipykernel_launcher.py [ARGUMENTS]
2023-03-03 15:29:01.750 
  command:

    streamlit run /usr/local/lib/python3.8/dist-packages/ipykernel_launcher.py [ARGUMENTS]


In [ ]:
!pip install easyocr


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install git+git://github.com/jaidedai/easyocr.git.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning git://github.com/jaidedai/easyocr.git. to /tmp/pip-req-build-h7r6enhr
  Running command git clone --filter=blob:none --quiet git://github.com/jaidedai/easyocr.git. /tmp/pip-req-build-h7r6enhr
  fatal: unable to connect to github.com:
  github.com[0: 140.82.112.4]: errno=Connection timed out

  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet git://github.com/jaidedai/easyocr.git. /tmp/pip-req-build-h7r6enhr did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet git://github.com/jaidedai/easyocr.git. /tmp/pip-req-build-h7r6enhr did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a proble